<h1>BatchEnsemble: An Alternative Approach To Efficient Ensemble and Lifelong Learning<h1/>

Yeming Wen, Dustin Tran & Jimmy Ba

<h2>Classification with Wide ResNet and CIFAR10<h2/>

In [2]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
m = nn.Softplus()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torchvision
import torchvision.transforms as transforms
data_dir = '/content/drive/My Drive/AALTO/cs4875-research/data/'
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Min-max scaling to [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

In [ ]:
class BlockEnsemble(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate, stride=1, num_models=4):
        super(BlockEnsemble, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = Ensemble_Conv2d(in_channels, out_channels, 3, stride=1, padding=1, first_layer=False, num_models=num_models, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(plaout_channelsnes)
        self.conv2 = Ensemble_Conv2d(out_channels, out_channels, 3, stride=stride, padding=1, first_layer=False, num_models=num_models, bias=True)
        self.num_models = num_models
        self.convs = [self.conv1, self.conv2]
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True),
                #Ensemble_Conv2d(in_planes, planes, 1, stride=stride, padding=0, first_layer=False, num_models=num_models, bias=True),
            )
    def update_indices(self, indices):
        for m_conv in self.convs:
            m_conv.update_indices(indices)

    def forward(self, x):
        curr_bs = x.size(0)
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.skip(x)

        return out